# [핀다] 태블로 sankey_data 업데이트_221031

### 오류 방지 코드

In [ ]:
# 패키지 설치 코드 
# !pip install warnings
# !pip install pathlib
# !pip install datetime
# !piA install oauth2client
# !pip install gspread
# !pip install openpyxl
# !pip install pyarrow
import warnings
warnings.filterwarnings("ignore")

- 위 패키지 설치 코드는 로컬에서 해당 파일을 최초로 실행할 때 한번만 실행해주시면 됩니다.
- 코드 실행 시 아래 이미지와 같은 경고창이 뜨는 것은 문제가 되지 않으니 무시해주세요.
    - <img src="https://i.ibb.co/JF0dpWY/requirement-alreay-satisfied.png" align="left">

- 이후 아래 이미지와 같은 형식의 오류가 발생할 경우 `!pip install 패키지명` 형식의 코드 작성하여 패키지 추가 설치해주세요.
    - <img src = 'https://ifh.cc/g/ymACFT.png' align='left'>

### 디렉토리 설정
- 아래 download_dir에 최종 결과파일을 다운받을 폴더 위치경로를 입력해주세요.
- 경로 주소는 '.../.../...' 형식을 따라주세요.

In [ ]:
import os

class directory:
    dir_list = os.getcwd().split('\\')
    dropbox_dir = '/'.join(dir_list[:dir_list.index('Dropbox (주식회사매드업)')+1])
    # 드롭박스 폴더 위치경로
    
    download_dir = dropbox_dir + '/광고사업부/데이터컨설팅/Tableau/result/핀다/sankey'
    # 최종 결과파일을 다운받을 폴더 위치경로

dr = directory()

### 날짜 설정
- 보통 작일자 기준으로 요청주신 기간의 데이터를 출력하게 됩니다.
- 날짜 조정이 필요한 경우 아래 변수 값을 수정하여 출력해주세요.

In [ ]:
import datetime

class report_date:
    today = datetime.date.today()
    # 오늘 날짜 ex> 2022-09-14
    
    # today = datetime.date(year = 2022, month = 8, day = 12)
    # 다른 일자를 기준으로 추출하고 싶은 경우 위 코드 수정하여 활용
    
    day_1 = today - datetime.timedelta(1)
    # 전일자 ex> 2022-09-13
    
    yearmonth = day_1.strftime('%Y%m')
    # 년월 ex> 202209

rdate = report_date()

### 구글 스프레드 시트 읽기
- 특정 구글 스프레드 시트를 읽어오기 위해 활용하는 메소드 모음입니다.
- dcteam@madup-355605.iam.gserviceaccount.com 계정에 참조하고자 하는 구글 시트 공유가 필요합니다.

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import time

class spreadsheet:
    def spread_document_read(self, url):
        token_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/token'
        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        json_file = token_dir + '/madup-355605-cd37b0ac201f.json'
        credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file, scope)
        gc = gspread.authorize(credentials)
        read = gc.open_by_url(url)
        print('스프레드시트 읽기 완료')
        return read
    
    def spread_sheet(self, doc, sheet_name, col_num=0, row_num=0):
        while True :
            try :
                data_sheet = doc.worksheet(sheet_name)
                data_sheet_read = data_sheet.get_all_values()
                result = pd.DataFrame(data_sheet_read, columns=data_sheet_read[row_num]).iloc[row_num+1:, col_num:]
            except :
                print('API 오류로 15초 후 다시 시도 합니다.')
                time.sleep(15)
                continue
            break
        return result

spreadsheet = spreadsheet()

### 코드
- 전처리 자동화를 위한 코드로 임의 수정을 하실 경우 복사본 생성하여 진행해주시길 바랍니다.
- 이외의 오류가 발생하거나 문제가 해결되지 않는 경우, 데컨팀에 문의주세요 :)

- 아래 시트에 정보를 우선 기입한 후 아래 코드를 실행하셔야 정상 동작됩니다.
    - https://docs.google.com/spreadsheets/d/1_-jgnU51smApXKQ4R8n0ek984s-4NQ1TfQefv_SbQKE/edit#gid=0

In [ ]:
class tableau_info():
    account_name = '핀다'
    result_name = 'finda'

tableau_info = tableau_info()

In [ ]:
import pyarrow as pa
import pyarrow.csv as pacsv
import pandas as pd
import os


prep_doc = spreadsheet.spread_document_read('https://docs.google.com/spreadsheets/d/1_-jgnU51smApXKQ4R8n0ek984s-4NQ1TfQefv_SbQKE/edit#gid=0')

def get_custom_sheet(doc, sheet_name):
    # 태블로 대시보드 컬럼 설정 불러오기
    sheet_data = spreadsheet.spread_sheet(doc, sheet_name)

    return sheet_data


def get_raw_df(apps_dir, is_organic, required_date):
    if is_organic == False:
        dtypes = {
            'attributed_touch_time': pa.string(),
            'attributed_touch_type': pa.string(),
            'install_time': pa.string(),
            'event_time': pa.string(),
            'event_name': pa.string(),
            'media_source': pa.string(),
            'adset': pa.string(),
            'ad': pa.string(),
            'campaign': pa.string(),
            'appsflyer_id': pa.string(),
            'is_retargeting': pa.string(),
            'event_value': pa.string(),
            'platform': pa.string()
        }
    else:
        dtypes = {
            'Attributed Touch Time': pa.string(),
            'Attributed Touch Type': pa.string(),
            'Install Time': pa.string(),
            'Event Time': pa.string(),
            'Event Name': pa.string(),
            'Media Source': pa.string(),
            'Campaign': pa.string(),
            'AppsFlyer ID': pa.string(),
            'Is Retargeting': pa.string(),
            'Event Value': pa.string(),
            'Platform': pa.string()
        }

    index_columns = list(dtypes.keys())
    convert_ops = pacsv.ConvertOptions(column_types=dtypes, include_columns=index_columns)
    ro = pacsv.ReadOptions(block_size=10 << 20)
    table_list = []

    date_check = required_date.strftime('%Y%m')
    start_date = required_date.replace(day=1).strftime('%Y%m%d')
    end_date = required_date.strftime('%Y%m%d')

    files = os.listdir(apps_dir)
    files = [f for f in files if '.csv' in f and str(f)[-12:-6] == date_check]
    raw_files = [f for f in files if
                 (int(str(f)[-12:-4]) >= int(start_date)) & (int(str(f)[-12:-4]) <= int(end_date))]

    for f in raw_files:
        temp = pacsv.read_csv(apps_dir + '/' + f, convert_options=convert_ops, read_options=ro)
        table_list.append(temp)
    table = pa.concat_tables(table_list)
    raw_df = table.to_pandas()
    if is_organic == False:
        raw_df['is_organic'] = False
    else:
        raw_df = raw_df.rename(columns=
                               {'Attributed Touch Time': 'attributed_touch_time',
                                'Attributed Touch Type': 'attributed_touch_type',
                                'Install Time': 'install_time',
                                'Event Time': 'event_time',
                                'Event Name': 'event_name',
                                'Media Source': 'media_source',
                                'Campaign': 'campaign', 'AppsFlyer ID': 'appsflyer_id',
                                'Is Retargeting': 'is_retargeting',
                                'Event Value': 'event_value',
                                'Platform': 'platform'})
        raw_df['is_organic'] = True

    return raw_df


### finda 추가 필터링 조건 ###
def finda_paid_prep(raw_df):
    raw_df.loc[raw_df['attributed_touch_type']!='click', 'is_organic'] = True
    raw_df.loc[(raw_df['platform']=='ios')&(raw_df['attributed_touch_time']==''), 'is_organic'] = False

    # 모비데이즈 데이터 예외처리
    raw_df_copy = raw_df.copy()
    raw_df_copy = raw_df_copy.loc[raw_df_copy['media_source']!='']
    raw_df_copy_dedup = raw_df_copy[['media_source', 'campaign', 'adset', 'ad']].drop_duplicates(['campaign', 'adset', 'ad'])
    raw_df_copy_dedup = raw_df_copy_dedup.rename(columns = {'media_source' : 'media_source_find'})

    raw_df_merge = raw_df.merge(raw_df_copy_dedup, on = ['campaign', 'adset', 'ad'], how = 'left')

    raw_df_merge.loc[raw_df_merge['media_source']=='', 'media_source'] = raw_df_merge['media_source_find']

    del raw_df_merge['media_source_find']
    return raw_df_merge
############################


def raw_data_concat(apps_paid_dir, apps_organic_dir, media_filter, required_date):
    if (apps_paid_dir != dr.dropbox_dir)&(apps_organic_dir != dr.dropbox_dir):
        paid_df = get_raw_df(apps_paid_dir, False, required_date)
        organic_df = get_raw_df(apps_organic_dir, True, required_date)
        ### finda 추가 필터링 ###
        paid_df = finda_paid_prep(paid_df)
        #######################
        raw_df = pd.concat([paid_df, organic_df], sort=False, ignore_index=True)
    elif (apps_paid_dir != dr.dropbox_dir)&(apps_organic_dir == dr.dropbox_dir):
        raw_df = get_raw_df(apps_paid_dir, False, required_date)
        ### finda 추가 필터링 ###
        raw_df = finda_paid_prep(raw_df)
        #######################
    elif (apps_paid_dir == dr.dropbox_dir)&(apps_organic_dir != dr.dropbox_dir):
        raw_df = get_raw_df(apps_organic_dir, True, required_date)
    else:
        raise Exception('드롭박스 내 appsflyer 데이터 위치를 입력 해주세요.')

    raw_df['is_retargeting'] = raw_df['is_retargeting'].str.lower()

    raw_df[['attributed_touch_time', 'install_time', 'event_time']] = raw_df[
        ['attributed_touch_time', 'install_time', 'event_time']].apply(pd.to_datetime)

    raw_df['click_date'] = raw_df['attributed_touch_time'].dt.date
    raw_df['click_hour'] = raw_df['attributed_touch_time'].dt.hour
    raw_df['click_weekday'] = raw_df['attributed_touch_time'].dt.weekday
    raw_df['event_date'] = raw_df['event_time'].dt.date
    raw_df['event_hour'] = raw_df['event_time'].dt.hour
    raw_df['event_weekday'] = raw_df['event_time'].dt.weekday
    raw_df['CTET'] = raw_df['event_time'] - raw_df['attributed_touch_time']

    raw_df = raw_df.sort_values(['event_time','attributed_touch_time'])
    raw_df.index = range(0, len(raw_df))

    # raw_df_dedup = raw_df.drop_duplicates(['event_time', 'event_name', 'appsflyer_id'], keep='last')
    raw_df_dedup = raw_df.loc[~(raw_df['media_source'].isin(media_filter))]

    return raw_df_dedup


def campaign_name_exception(prep_doc, raw_df):
    ### finda 추가 필터링 조건 ###
        # 캠페인명 수정
    raw_df.loc[raw_df['media_source'] != 'moloco_int', 'campaign'] = raw_df['campaign'].apply(lambda x: x.replace('MobidaysAgency_', 'Madit_'))
    raw_df.loc[raw_df['media_source'] != 'moloco_int', 'campaign'] = raw_df['campaign'].apply(lambda x: x.replace('Mobi_', 'Madit_'))
    raw_df.loc[raw_df['media_source'] == 'cauly_int', 'campaign'] = raw_df['campaign'].apply(lambda x: 'Madit_CAULY_LOAN_RT_AOS_CPC_220714' if x.startswith('99') else x)
    raw_df.loc[raw_df['media_source'] == 'tnk_int', 'campaign'] = 'Madit_TNK_LOAN_NU_AOS_REWARD-CPA_220715'
        # 특정 캠페인 구분 organic으로 변경
    raw_df.loc[(raw_df['media_source'] == 'kakao_int')&(raw_df['campaign']=='kakao_biz_loancontract'), 'is_organic'] = True
    raw_df.loc[(raw_df['media_source'] == 'kakao')&(raw_df['campaign'] != 'talk'), 'is_organic'] = True
    ############################

    campaign_mapping = get_custom_sheet(prep_doc, tableau_info.account_name)
    media_source = campaign_mapping.media_source.unique()
    campaign_dict = {}
    for source in media_source:
        campaign_temp = campaign_mapping.loc[campaign_mapping['media_source']==source][['before_campaign','after_campaign']].set_index('before_campaign').to_dict()
        campaign_dict[source] = campaign_temp['after_campaign']

    for media in campaign_dict.keys():
        raw_df.loc[raw_df['media_source'] == media, 'campaign'] = raw_df['campaign'].apply(
        lambda x: campaign_dict[media][x] if x in campaign_dict[media].keys() else x)

    return raw_df


def sankey_data_prep(prep_doc, required_date, file_name):
    sheet_data = get_custom_sheet(prep_doc, '가공조건')
    info_data = sheet_data.loc[sheet_data['광고주'] == tableau_info.account_name]

    apps_paid_dir = dr.dropbox_dir + info_data['paid_raw 데이터 위치'].iloc[0]
    apps_organic_dir = dr.dropbox_dir + info_data['organic_raw 데이터 위치'].iloc[0]
    if info_data['제외 매체'].iloc[0] == '':
        media_filter = []
    else:
        media_filter = info_data['제외 매체'].iloc[0].strip().replace(', ',',').split(',')
    raw_df = raw_data_concat(apps_paid_dir, apps_organic_dir, media_filter, required_date)

    if info_data['캠페인명 가공여부'].iloc[0] == 'TRUE':
        raw_df_exception = campaign_name_exception(prep_doc, raw_df)
    else:
        raw_df_exception = raw_df

    target_event_list = info_data['타겟 이벤트'].iloc[0].strip().replace(', ',',').split(',')
    event_data = raw_df_exception.loc[raw_df_exception['event_name'].isin(target_event_list)]

    event_data.loc[pd.isnull(event_data['attributed_touch_time']), 'attributed_touch_time'] = event_data['install_time']
    event_data['click_date'] = event_data['attributed_touch_time'].dt.date
    event_data['click_weekday'] = event_data['attributed_touch_time'].dt.weekday
    event_data = event_data.loc[event_data['attributed_touch_type']!='impression']


    event_data['CTET'] = (event_data['event_time'] - event_data['attributed_touch_time']).apply(
        lambda x: x.total_seconds() / 86400)
    event_data['CTIT'] = (event_data['install_time'] - event_data['attributed_touch_time']).apply(
        lambda x: x.total_seconds() / 86400)
    event_data['ITET'] = (event_data['event_time'] - event_data['install_time']).apply(lambda x: x.total_seconds() / 86400)

    ctet_limit = info_data['CTET 기여기간'].iloc[0]
    ctit_limit = info_data['CTIT 기여기간'].iloc[0]
    itet_limit = info_data['ITET 기여기간'].iloc[0]

    if ctet_limit != '':
        event_data = event_data.loc[event_data['CTET'] < float(ctet_limit)]
    if ctit_limit != '':
        event_data = event_data.loc[event_data['CTIT'] < float(ctit_limit)]
    if itet_limit != '':
        event_data = event_data.loc[event_data['ITET'] < float(itet_limit)]

    event_data = event_data.loc[~event_data['media_source'].isin(['', 'restricted'])]
    event_data['Cnt'] = 1
    event_data['click_month'] = event_data['attributed_touch_time'].dt.month
    event_data['event_month'] = event_data['event_time'].dt.month
    event_data['UA / RE'] = event_data['is_retargeting'].apply(lambda x : 'RE' if x == 'true' else 'UA')

    event_data_filtered = event_data[['UA / RE', 'media_source', 'click_month', 'click_date', 'click_weekday',
                                      'event_month','event_date','event_weekday',
                                      'event_name','platform','appsflyer_id','Cnt']]

    event_data_pivot = event_data_filtered.pivot_table(index=['UA / RE', 'media_source', 'click_month', 'click_date', 'click_weekday',
                                                              'event_month','event_date','event_weekday','platform','event_name'],
                                                       aggfunc='sum').reset_index()


    event_data_pivot = event_data_pivot.sort_values(['event_month','click_month', 'event_weekday','click_weekday'])

    weekday_dict = {0: '월요일',
                    1: '화요일',
                    2: '수요일',
                    3: '목요일',
                    4: '금요일',
                    5: '토요일',
                    6: '일요일'}
    event_data_pivot['click_weekday'] = event_data_pivot['click_weekday'].apply(lambda x: weekday_dict.get(x))
    event_data_pivot['event_weekday'] = event_data_pivot['event_weekday'].apply(lambda x: weekday_dict.get(x))

    event_data_pivot['click_month'] = event_data_pivot['click_month'].apply(lambda x: str(x) + "월")
    event_data_pivot['event_month'] = event_data_pivot['event_month'].apply(lambda x: str(x) + "월")
    event_data_pivot.to_excel(dr.download_dir + '/' + file_name, index=False, encoding='utf-8-sig')
    print('download successful')

def union_sandkey_data(required_date, file_name):
    date_check = required_date.strftime('%Y')
    total_df = pd.DataFrame()

    files = os.listdir(dr.download_dir)
    files = [f for f in files if '.xlsx' in f and str(f)[-11:-7] == date_check]
    raw_files = [f for f in files]
    for file in raw_files:
        temp = pd.read_excel(dr.download_dir + '/' + file)
        total_df = pd.concat([total_df, temp], axis=0)

    total_df.to_excel(dr.download_dir + '/' + file_name, index=False, encoding='utf-8-sig')
    print('union successful')


file_name = f'{tableau_info.result_name}_event_flow_data_{rdate.yearmonth}.xlsx'
sankey_data_prep(prep_doc, rdate.day_1, file_name)
union_sandkey_data(rdate.day_1, f'{tableau_info.result_name}_event_flow_data_{rdate.day_1.year}_total.xlsx')

---